In [5]:
import nibabel as nib
import numpy as np
import os
import glob
from sklearn.linear_model import LinearRegression
#import plotnine as p9
import matplotlib.pyplot as plt
#%matplotlib inline
import csv
import statsmodels.api as sm
from scipy import stats
#from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
#import import_ipynb
#from Predistim_Library import *

###  Lecture fichier Quality Check

In [6]:
#CQData = pd.read_csv(os.path.join(STUDY_PATH,'CQData20200406.csv'),index_col=0,sep=';')
CQData = pd.read_excel(CQ_FILE,index_col=0)
print("n = ",len(CQData))
CQData.columns

NameError: name 'pd' is not defined

### Lecture du fichier de données genomique

In [ ]:
#GenomicData = pd.read_excel(os.path.join(STUDY_PATH,'SNPs_Predistim.xlsx'),usecols=["NUM_PAT","SNP_ACSL4","SNP_GPx4","cluster_SNP"],index_col=0)
GenomicData = pd.read_excel(SNP_FILE,index_col=0,converters = {'PatientNB': int})
print("n = ",len(GenomicData))

### Lecture du fichier de données démographique (extraction du 22/06/2020)

In [5]:
ClinicalData = pd.read_excel(CLI_FILE,index_col=1)
#Cli_Gen_data=Cli_Gen_data.set_index(['PatientNB'])
print("n = ",len(ClinicalData))
print(ClinicalData.columns)

XLRDError: Excel xlsx file; not supported

### CQ

In [5]:
CQ_DB=set(CQData.index)
Full_DB=set(ClinicalData.set_index(['PatientID']).index)
print(len(CQ_DB-Full_DB),CQ_DB-Full_DB)
print(len(Full_DB-CQ_DB),Full_DB-CQ_DB)

print(len(CQ_DB | Full_DB))

T1_DB=set(CQData.loc[CQData['QC_3DT1']>1].index)
print(len(T1_DB))

TE_DB=set(CQData.loc[CQData['QC_3DMULTI']>1].index)
print(len(TE_DB))

print(len(T1_DB & TE_DB))

14 {'10037LJ', '10036FM', '19031CF', '13023DJ', '13020BE', '10044HF', '19047MA', '13006ZP', '13005VA', '09026PF', '13015IR', '13026RD', '02051BG', '04010BC'}
51 {'15019CJ', '15023FP', '15024MD', '15012LM', '07034HC', '17063EE', '15018HJ', '13005VM', '02051GB', '20004DG', '13023', '09005SN', '19031GF', '15035CJ', '09003ME', '10037FM', '15014BG', '15009PJ', '14051AV', '15011MP', '11005DJ', '09006JY', '17036GD', '13020', '13015LR', '09015CA', '15030LN', '08027BB', '11002HD', '15020FI', '15005CB', '09009GM', '15032DM', '05008DM', '11003DM', '15022SJ', '15033CC', '17074VG', '10036LJ', '16027DB', '13026', '15031BF', '15006BB', '01036LD', '17012BF', '08021MM', '15004MJ', '17007JY', '15015BM', '15003LJ', '15021EA'}
848
678
451
449


In [6]:
file_list=glob.glob(os.path.join(STUDY_PATH,'3DT1','*','3DT1_*.nii.gz'))
file_list = [os.path.dirname(w)for w in file_list]
file_list = [w.replace(os.path.join(STUDY_PATH,'3DT1/'),'') for w in file_list]
T1_DB-set(file_list)

set()

In [7]:
set(file_list)-T1_DB

{'13026RD', '13045MS', '13053RS', '13056OA'}

In [12]:
tmp=[]
file_list=glob.glob(os.path.join(STUDY_PATH,'HCP','*','T1w','T1w.nii.gz'))
for file in file_list :
    tmp.append(file.split('/')[6])
T1_FILE_DB=set(tmp)
print(len(T1_FILE_DB),T1_DB-T1_FILE_DB)

tmp=[]
file_list=glob.glob(os.path.join(STUDY_PATH,'volbrain','native_*','native_lab_n_mmni_*.nii'))
for file in file_list :
    tmp.append(file.split('/')[6].split('_')[1])
VB_FILE_DB=set(tmp)
print(len(VB_FILE_DB),T1_DB-VB_FILE_DB)

tmp=[]
file_list=glob.glob(os.path.join(STUDY_PATH,'3DT1','*','FLASH_RN_L*.nii.gz'))
for file in file_list :
    tmp.append(file.split('/')[6])
SUBC_FILE_DB=set(tmp)
print(len(SUBC_FILE_DB),T1_DB-SUBC_FILE_DB)

tmp=[]
file_list=glob.glob(os.path.join(STUDY_PATH,'HCP','*','T1w','aparc+aseg.nii.gz'))
for file in file_list :
    tmp.append(file.split('/')[6])
APARC_ASEG_FILE_DB=set(tmp)
print(len(APARC_ASEG_FILE_DB),T1_DB-APARC_ASEG_FILE_DB)

tmp=[]
file_list=glob.glob(os.path.join(STUDY_PATH,'2020-04-15','*','*','M00','freesurfer_aparc+aseg.nii.gz'))
#print(len(file_list))
for file in file_list :
    tmp.append(file.split('/')[7])

APARC_ASEG_FILE_DB2=set(tmp)
print(len(APARC_ASEG_FILE_DB2),T1_DB-APARC_ASEG_FILE_DB2)

646 {'02059WC', '14004IJ', '02058NP', '02080RM', '02051BG', '02075LP', '02053MC', '02063VC', '02064CA', '02083SG', '02057DH', '02081HC', '02040DC', '02078RM', '02079MZ', '02085PP', '02046CA', '02084DI', '02082SR', '02089SL', '02067BE', '02073MD', '02069VF', '02071BM', '02045PJ', '02070DJ', '02087TD', '02090DA'}
661 {'02059WC', '14004IJ', '02058NP', '02080RM', '02051BG', '02075LP', '02053MC', '02063VC', '02064CA', '02083SG', '02057DH', '02081HC', '02040DC', '02078RM', '02079MZ', '02085PP', '02046CA', '02084DI', '02082SR', '02089SL', '02067BE', '02073MD', '02069VF', '02071BM', '02045PJ', '02070DJ', '02087TD', '02090DA'}
639 {'10028GJ', '10006AP', '02059WC', '14004IJ', '02058NP', '02080RM', '02051BG', '02075LP', '02053MC', '02063VC', '02064CA', '02083SG', '02057DH', '10035MP', '02081HC', '02040DC', '02078RM', '02079MZ', '10020NJ', '02085PP', '02046CA', '02084DI', '02082SR', '02089SL', '02067BE', '02073MD', '02069VF', '02071BM', '02045PJ', '02070DJ', '02087TD', '02090DA'}
640 {'02059WC', '

In [351]:
set(ClinicalData.index)-set(GenomicData.index)

{10031,
 20031,
 40001,
 40002,
 40003,
 40004,
 40005,
 40006,
 40007,
 40008,
 40009,
 40011,
 40012,
 40013,
 40014,
 40015,
 40016,
 40017,
 50004,
 50008,
 50014,
 50015,
 50016,
 50020,
 50025,
 50026,
 50037,
 60001,
 60068,
 60084,
 60090,
 60091,
 60092,
 60093,
 80001,
 80002,
 80003,
 80004,
 80005,
 80006,
 80007,
 80008,
 80009,
 80010,
 80011,
 80012,
 80013,
 80014,
 80015,
 80016,
 80017,
 80018,
 80019,
 80020,
 80021,
 80022,
 80023,
 80024,
 80025,
 80026,
 80027,
 80028,
 80029,
 80030,
 80031,
 80032,
 80033,
 80034,
 80035,
 80036,
 80037,
 80038,
 80039,
 80040,
 80041,
 80042,
 90001,
 90002,
 90003,
 90004,
 90005,
 90006,
 90007,
 90008,
 90009,
 90010,
 90011,
 90012,
 90013,
 90014,
 90015,
 90016,
 90017,
 90018,
 90019,
 90020,
 90021,
 90022,
 90023,
 130001,
 130002,
 130003,
 130004,
 130005,
 130007,
 130008,
 130009,
 130010,
 130011,
 130012,
 130013,
 130014,
 130015,
 130016,
 130017,
 130018,
 130019,
 130020,
 130021,
 130022,
 130023,
 130024,
 

In [352]:
#list_Cli_Gen=ClinicalData.index.intersection(GenomicData.index)
#ClinicalData=ClinicalData.loc[list_Cli_Gen]
#GenomicData=GenomicData.loc[list_Cli_Gen]
Cli_Gen_data=ClinicalData.join(GenomicData)
Cli_Gen_data=Cli_Gen_data.set_index(['PatientID'])

In [ ]:
SUBJ='01011BB'
PARA='R2'
file=os.path.join(STUDY_PATH,PARA,SUBJ,PARA+'_lin_3DT1.nii.gz') 
img_data=np.asanyarray(nib.load(file).dataobj)
name_struct='r_FLASH_SN'
#for cpt2,name_struct in enumerate(['r_FLASH_RN','r_FLASH_SN','r_QSM_GPe','r_QSM_GPi','r_FLASH_STh','r_MP2RAGE_STR']):
struct_L_ = nib.load(os.path.join(STUDY_PATH,'R2',SUBJ,name_struct+'_L_'+SUBJ+'_on_3DT1.nii.gz'))
struct_R_ = nib.load(os.path.join(STUDY_PATH,'R2',SUBJ,name_struct+'_R_'+SUBJ+'_on_3DT1.nii.gz'))
struct_data=np.asanyarray(struct_R_.dataobj)+np.asanyarray(struct_L_.dataobj)
tmp = (struct_data>0)

struct = nib.load(os.path.join(STUDY_PATH,'volbrain','native_'+SUBJ,'native_lab_'+SUBJ+'_resliced.nii.gz'))
struct_data=np.asanyarray(struct.dataobj)                    
tmp = ((struct_data==3) | (struct_data==4))

plot=p9.ggplot()+ p9.geom_histogram(p9.aes(img_data[tmp>0]),binwidth=1)+p9.xlab("Caudate")
plot.draw()

In [47]:
plot.save(os.path.join(STUDY_PATH,'Dist_Caudate.jpg'))

/home/romain/.local/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/romain/.local/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /NAS/deathrow/protocoles/predistim/Dist_Caudate.jpg


In [16]:
os.path.join(STUDY_PATH,'R2',SUBJ,name_struct+'_L_'+SUBJ+'_on_3DT1.nii.gz')

'/NAS/deathrow/protocoles/predistim/R2/01011BB/r_FLASH_SN_L_01011BB_on_3DT1.nii.gz'

# Pour les étudiants de LAS

In [102]:
ClinicalData = pd.read_csv(os.path.join(STUDY_PATH,'DataCli_LAS.csv'),index_col=0,sep=',')
#Cli_Gen_data=Cli_Gen_data.set_index(['PatientNB'])
#ClinicalData['Sexe']= ClinicalData['Sexe'].astype(bool)
print("n = ",len(ClinicalData))
print(ClinicalData.columns)
print(ClinicalData.info())
print(ClinicalData.describe())


n =  816
Index(['Age', 'BMI', 'Sexe'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 01001SR to nan
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     813 non-null    float64
 1   BMI     772 non-null    float64
 2   Sexe    812 non-null    float64
dtypes: float64(3)
memory usage: 25.5+ KB
None
              Age         BMI        Sexe
count  813.000000  772.000000  812.000000
mean    59.881919   25.218912    1.350985
std      8.397210    4.400802    0.477572
min      0.000000   15.000000    1.000000
25%     56.000000   22.000000    1.000000
50%     61.000000   25.000000    1.000000
75%     66.000000   28.000000    2.000000
max     73.000000   43.000000    2.000000


In [103]:
VolData = pd.read_excel(os.path.join(STUDY_PATH,'volCranien.xls'),index_col=0)
#Cli_Gen_data=Cli_Gen_data.set_index(['PatientNB'])
print("n = ",len(VolData))
print(VolData.info())

n =  679
<class 'pandas.core.frame.DataFrame'>
Index: 679 entries, 01001SR to 20029DA
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Vol_(mm3)  678 non-null    float64
dtypes: float64(1)
memory usage: 10.6+ KB
None


In [104]:
myDataFrame=VolData.merge(ClinicalData,left_index=True,right_index=True)
myDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 652 entries, 01001SR to 20029DA
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Vol_(mm3)  651 non-null    float64
 1   Age        651 non-null    float64
 2   BMI        615 non-null    float64
 3   Sexe       650 non-null    float64
dtypes: float64(4)
memory usage: 25.5+ KB


In [105]:
myDataFrame.to_csv(os.path.join(STUDY_PATH,'df_Predisitim_LAS.csv'))

In [106]:
test = pd.read_csv(os.path.join(STUDY_PATH,'df_Predisitim_LAS.csv'))
test.describe()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  652 non-null    object 
 1   Vol_(mm3)   651 non-null    float64
 2   Age         651 non-null    float64
 3   BMI         615 non-null    float64
 4   Sexe        650 non-null    float64
dtypes: float64(4), object(1)
memory usage: 25.6+ KB


In [107]:
myDataFrame.columns

Index(['Vol_(mm3)', 'Age', 'BMI', 'Sexe'], dtype='object')

In [108]:
myDataFrame.groupby(['Sexe']).mean()['BMI']

Sexe
1.0    25.575000
2.0    24.367442
Name: BMI, dtype: float64

In [109]:
myDataFrame.groupby(['Sexe'])['BMI'].describe()

,count,mean,std,min,25%,50%,75%,max
Sexe,,,,,,,,
1.0,400.0,25.575000,4.090379,17.0,23.0,25.0,28.0,43.0
2.0,215.0,24.367442,4.877936,15.0,21.0,24.0,27.0,42.0


In [110]:
myDataFrame.groupby(['Sexe'])['Vol_(mm3)'].describe()

,count,mean,std,min,25%,50%,75%,max
Sexe,,,,,,,,
1.0,422.0,1.525607e+06,123565.905671,908043.0,1443707.0,1519454.0,1603618.0,1979771.0
2.0,227.0,1.330809e+06,103884.725451,1079919.0,1251250.0,1333266.0,1413264.0,1566651.0


In [89]:
hist=myDataFrame['Vol_(mm3)'].hist(by=myDataFrame['Sexe'])
plt.savefig("hist_LAS.png", bbox_inches='tight', dpi=100)